# Code #

In [2]:
import urllib.request
import json
import pandas as pd

In [3]:
df_factoryID=pd.read_csv("https://raw.githubusercontent.com/wichukornk/FEC/main/factoryID2.txt",header=None)
df_factoryID[0]=df_factoryID[0].str.replace("<option value=\"","")
df_factoryID[0]=df_factoryID[0].str.replace("</option>","")
df_factoryID[0]=df_factoryID[0].str.replace("\">",",")
df_factoryID=df_factoryID[0].str.split(pat=",",expand=True)
df_factoryID.head()

,0,1
0,F00000000000039,mmm
1,F00000000000040,ไม่ใช่ใครสักคน
2,F00000000000041,factory18
3,F00000000000042,12
4,F00000000000043,โรงงาน01


## Electric table ##

In [ ]:
def getElecData(facID,sysType):
  url_elec="http://54.254.220.213/besten/backend/web/index.php?r=factories%2Freport&rid="+str(sysType)+"&fid="+facID+"&tid=2&user=admin@diw.com&pass=123456"
  response = urllib.request.urlopen(url_elec)
  data = json.loads(response.read())
  df1=pd.DataFrame(data)
  col_list=['feID', 'feName', 'elec_price', 'feType', 'feType2', 'feSize', 'feUnit','feHours', 'nameplate', 'cal_nameplate', 'nameplate_unit', 'np_d','pot_mj', 'savebaht']
  df=pd.DataFrame(columns=col_list)
  for i in range(len(df1)):
    df2=df1.iloc[i,0]
    df2=pd.DataFrame(df2,index=[i])
    df=df.append(df2)
  df['facID']=facID
  df['equipType']=sysType
  col_list=['facID','equipType']+col_list
  return df.loc[:,col_list]

df_elec_main=pd.DataFrame()
df_elec_sup=pd.DataFrame()

for j in df_factoryID.index: 
  df_elec_main_tod=getElecData(facID=df_factoryID.loc[j,0],sysType=3)
  df_elec_main=df_elec_main.append(df_elec_main_tod)
  df_elec_sup_tod=getElecData(facID=df_factoryID.loc[j,0],sysType=4)
  df_elec_sup=df_elec_sup.append(df_elec_sup_tod)
  #print(df_elec_sup_tod.head(3))

df_elec=df_elec_main.append(df_elec_sup)
df_elec['facName']=df_elec['facID'].map(dict(zip(df_factoryID[0],df_factoryID[1])))
df_elec['sysCate']=df_elec['equipType'].astype('str').map({"3":"Main electric equipment","4":"Support electric equipment"})
df_elec.head()

,facID,equipType,feID,feName,elec_price,feType,feType2,feSize,feUnit,feHours,nameplate,cal_nameplate,nameplate_unit,np_d,pot_mj,savebaht,facName,sysCate
0,F00000000000002,3,1,เครื่องฉีดหลอดพลาสติก,4.00,มอเตอร์,4 ขั้ว,360.00,kW,6336.00,NaN,92.40,%,ผ่าน,0,0,บริษัท หาดทิพย์ จำกัด (มหาชน),Main electric equipment
1,F00000000000002,3,2,เครื่องอัดลม 8 Bar - 1,4.00,มอเตอร์,4 ขั้ว,45.00,kW,6336.00,94.7,92.40,%,ไม่ผ่าน,7494.34,29977.4,บริษัท หาดทิพย์ จำกัด (มหาชน),Main electric equipment
2,F00000000000002,3,3,เครื่องอัดลม 8 Bar - 2,4.00,มอเตอร์,4 ขั้ว,45.00,kW,6336.00,94.7,92.40,%,ไม่ผ่าน,7494.34,29977.4,บริษัท หาดทิพย์ จำกัด (มหาชน),Main electric equipment
3,F00000000000002,3,4,เครื่องอัดลม 8 Bar - 3,4.00,มอเตอร์,4 ขั้ว,45.00,kW,6336.00,94.7,92.40,%,ไม่ผ่าน,7494.34,29977.4,บริษัท หาดทิพย์ จำกัด (มหาชน),Main electric equipment
4,F00000000000002,3,5,เครื่องเป่าขวด PET,4.00,มอเตอร์,4 ขั้ว,240.00,kW,6336.00,NaN,92.40,%,ผ่าน,0,0,บริษัท หาดทิพย์ จำกัด (มหาชน),Main electric equipment


In [ ]:
df_elec.to_gbq(destination_table='FEC.electric_equipment',project_id='introcloud01',if_exists='replace')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=7xH1aJCv9dveVL1uGLicE4GONgorZF&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g77bXfkSS8J7KvbmvV-8R2k-dC1GxdOmlH_JzSBMLiFLK_s6BoLkvc


1it [00:03,  3.36s/it]


## Thermal table ##

In [ ]:
def getThermData(facID,sysType):
  url_therm="http://54.254.220.213/besten/backend/web/index.php?r=factories%2Freport&rid="+str(sysType)+"&fid="+facID+"&tid=2&user=admin@diw.com&pass=123456"
  response = urllib.request.urlopen(url_therm)
  data = json.loads(response.read())
  df1=pd.DataFrame(data)
  col_list=['feID', 'feName', 'feType', 'feType2', 'feSize', 'feUnit', 'feHours','fuelType', 'fuelTypeUnit', 'feFuelY', 'feFuelPrice', 'nameplate','cal_nameplate', 'nameplate_unit', 'np_d', 'pot_mj', 'savefuel','savebaht']
  df=pd.DataFrame(columns=col_list)
  for i in range(len(df1)):
    df2=df1.iloc[i,0]
    df2=pd.DataFrame(df2,index=[i])
    df=df.append(df2)
  df['facID']=facID
  df['equipType']=sysType
  col_list=['facID','equipType']+col_list
  return df.loc[:,col_list]

df_therm_main=pd.DataFrame()
df_therm_sup=pd.DataFrame()

for j in df_factoryID.index :
  df_therm_main_tod=getThermData(facID=df_factoryID.loc[j,0],sysType=5)
  df_therm_main=df_therm_main.append(df_therm_main_tod)
  df_therm_sup_tod=getThermData(facID=df_factoryID.loc[j,0],sysType=6)
  df_therm_sup=df_therm_sup.append(df_therm_sup_tod)
  #print(df_therm_sup_tod.head(3))
  #print(df_factoryID.loc[j,1],len(df_therm_main_tod)+len(df_therm_sup_tod))

df_therm=df_therm_main.append(df_therm_sup)
df_therm['facName']=df_therm['facID'].map(dict(zip(df_factoryID[0],df_factoryID[1])))
df_therm['sysCate']=df_therm['equipType'].astype('str').map({"5":"Main thermal equipment","6":"Support thermal equipment"})
df_therm.head()

,facID,equipType,feID,feName,feType,feType2,feSize,feUnit,feHours,fuelType,fuelTypeUnit,feFuelY,feFuelPrice,nameplate,cal_nameplate,nameplate_unit,np_d,pot_mj,savefuel,savebaht,facName,sysCate
0,F00000000000002,6,4074,GETABEC,หม้อน้ำและหม้อน้ำร้อน,หม้อน้ำที่ใช้น้ำมันเป็นเชื้อเพลิง,5.00,ton/hr,6336.00,น้ำมันเตา (A),litre,370100.00,15.00,85,85.00,%,ผ่าน,0,None,0,บริษัท หาดทิพย์ จำกัด (มหาชน),Support thermal equipment
0,F00000000000005,6,3615,CIDAR ENGINEERING,หม้อน้ำและหม้อน้ำร้อน,หม้อน้ำที่ใช้น้ำมันเป็นเชื้อเพลิง,5.00,ton/hr,3000.00,น้ำมันดีเซล,litre,None,30.00,85,85.00,%,ผ่าน,0,None,0,บริษัท ชายแดนใต้ฟู๊ดโพรเซสซิ่ง จำกัด,Support thermal equipment
0,F00000000000008,6,3616,MD Boiler,หม้อน้ำและหม้อน้ำร้อน,หม้อน้ำที่ใช้แก๊สเป็นเชื้อเพลิง,1.00,ton/hr,2400.00,LPG,kg,None,11.46,80,92.00,%,ผ่าน,0,None,0,บริษัท จ่าวิรัช ฟู้ด จำกัด,Support thermal equipment
0,F00000000000010,6,3617,MIURA,หม้อน้ำและหม้อน้ำร้อน,หม้อน้ำที่ใช้น้ำมันเป็นเชื้อเพลิง,0.75,ton/hr,7200.00,น้ำมันดีเซล,litre,None,30.00,85,90.00,%,ผ่าน,0,None,0,บริษัท เอสเซนทรา จำกัด,Support thermal equipment
1,F00000000000010,6,3618,K.Boiler,หม้อน้ำและหม้อน้ำร้อน,หม้อน้ำที่ใช้แก๊สเป็นเชื้อเพลิง,2.00,ton/hr,7200.00,ขยะ,kg,None,0.00,80,90.00,%,ผ่าน,0,None,0,บริษัท เอสเซนทรา จำกัด,Support thermal equipment


In [ ]:
df_therm.to_gbq(destination_table='FEC.thermal_equipment',project_id='introcloud01',if_exists='replace')

1it [00:04,  4.91s/it]
